# Weekly project

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ)

In [4]:
import numpy as np
import cv2 as cv2
from numpy.linalg import inv, pinv
import matplotlib.pyplot as plt
import time as t
from helpers import *

def extract_keypoints_surf(img1, img2, K, baseline):
    """
    use surf to detect keypoint features
    remember to include a Lowes ratio test
    """
    # SURF IS DEAD!
    #surf = cv2.xfeatures2d.SURF_create()

    # 1 SIFT
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # 2 KNN match with two (best and second best)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # 3 Ratio test!
    kp1_indexes = []
    kp2_indexes = []
    for m,n in matches:
        if m.distance < 0.70*n.distance:
            kp1_indexes.append(m.queryIdx)
            kp2_indexes.append(m.trainIdx)

    # 4 extract the keypoints of the "good" matches
    kp1 = np.asarray(kp1)
    kp2 = np.asarray(kp2)
    match_points1 = [p.pt for p in kp1[kp1_indexes]]
    match_points2 = [p.pt for p in kp2[kp2_indexes]]

    p1 = np.array(match_points1).astype(np.float32)
    p2 = np.array(match_points2).astype(np.float32)
    ##############################
    ##### Do Triangulation #######
    ##############################
    #project the feature points to 3D with triangulation
    
    #projection matrix for Left and Right Image
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
    M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

    p1_flip = np.vstack((p1.T, np.ones((1, p1.shape[0]))))
    p2_flip = np.vstack((p2.T, np.ones((1, p2.shape[0]))))

    P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

    # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
    P = P / P[3]
    land_points = P[:3]

    return land_points.T, p1

def featureTracking(prev_img, next_img, prev_points, world_points):
    """
    Use OpenCV to find the prev_points from the prev_img in the next_img
    Remember to remove points that could not be found from prev_points, next_points, and world_points
    hint: status == 1
    """
    params = dict(winSize=(21, 21), # Window size of LK
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    # 1 Sparse optical flow: using LK (Lucas Kanada)
    #print(prev_img.shape)
    #print(type(prev_img))
    #print(next_img.shape)
    #print(type(next_img))
    #print(prev_points.shape)
    #print(type(prev_points))
    #print("---------------------------------------------")


    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_points, None, **params)
    
    # 2 Remove points that was not detected by both images, from 2D and 3D!
    next_points = next_points[np.where(np.any(status==1, axis=1))[0], :]
    world_points = world_points[np.where(np.any(status==1, axis=1))[0], :]
    prev_points = prev_points[np.where(np.any(status==1, axis=1))[0], :]

    #landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, curImage, reference_2D,landmark_3D)
    return world_points, prev_points, next_points

def playImageSequence(left_img, right_img, K):

    baseline = 0.54 # translation between first two images!

    ##### ################################# #######
    ##### Get 3D points Using Triangulation #######
    ##### #########################################
    """
    Implement step 1.2 and 1.3
    Store the features in 'reference_2D' and the 3D points (landmarks) in 'landmark_3D'
    hint: use 'extract_keypoints_surf' above
    """
    landmark_3D, reference_2D = extract_keypoints_surf(left_img, right_img, K, baseline)

    # reference
    reference_img = left_img

    # Groundtruth for plot
    truePose = getTruePose()
    traj = np.zeros((600, 600, 3), dtype=np.uint8)
    maxError = 0

    # The original camerar projection matrix
    M_old = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))

    for i in range(0, 1400):
        print('image: ', i)
        curImage = getLeftImage(i)
        #curImage_R = getRightImage(i) # In 2D-3D we only need one new frame
        print("-------------------------------------------------")
        print(landmark_3D)
        print("-------------------------------------------------")

        ##### ############################################################# #######
        ##### Calculate 2D and 3D feature correspndances in t=T-1, and t=T  #######
        ##### #####################################################################
        """
        Implement step 2.2)
        Remember this is a part of a loop, so the initial features are already
        provided in step 1)-1.3) outside the loop in 'reference_2D' and 'landmark_3D'
        """
        landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, 
                                                                      curImage, 
                                                                      reference_2D,
                                                                      landmark_3D)
        
        ##### ################################# #######
        ##### Calculate relative pose using PNP #######
        ##### #########################################
        """
        Implement step 2.3)
        """
        _, rvec, tvec, _ = cv2.solvePnPRansac(landmark_3D, tracked_2Dpoints, K, distCoeffs=None)

        ##### ####################################################### #######
        ##### Get Pose and Tranformation Matrix in world coordionates #######
        ##### ###############################################################
        rot, _ = cv2.Rodrigues(rvec)
        tvec = -rot.T.dot(tvec)  # coordinate transformation, from camera to world. What is the XYZ of the camera wrt World
        inv_transform = np.hstack((rot.T, tvec))  # inverse transform. A tranform projecting points from the camera frame to the world frame

        ##### ################################# #######
        ##### Get 3D points Using Triangulation #######
        ##### #########################################
        # re-obtain the 3D points
        """
        Implement step 2.4)
        """
        # Find completely new keypoints!
        # 1 SIFT
        sift = cv2.SIFT_create()
        kp1, des1 = sift.detectAndCompute(reference_img, None)
        kp2, des2 = sift.detectAndCompute(curImage, None)

        # 2 KNN match with two (best and second best)
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)

        # 3 Ratio test!
        kp1_indexes = []
        kp2_indexes = []
        for m,n in matches:
            if m.distance < 0.70*n.distance:
                kp1_indexes.append(m.queryIdx)
                kp2_indexes.append(m.trainIdx)

        # 4 extract the keypoints of the "good" matches
        kp1 = np.asarray(kp1)
        kp2 = np.asarray(kp2)
        match_points1 = [p.pt for p in kp1[kp1_indexes]]
        match_points2 = [p.pt for p in kp2[kp2_indexes]]

        old_2D = np.array(match_points1).astype(np.float32)
        new_2D = np.array(match_points2).astype(np.float32)

        #################
        ###Triangulate###
        #################
        M_new = K.dot(inv_transform) # Projection matrix for new image

        # Transpose and homogenousify it
        reference_flip = np.vstack((old_2D.T, np.ones((1, old_2D.shape[0]))))
        tracked_flip = np.vstack((new_2D.T, np.ones((1, new_2D.shape[0]))))

        P = cv2.triangulatePoints(M_old, M_new, reference_flip[:2], tracked_flip[:2])
        M_old = M_new.copy()
        # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
        P = P / P[3]

        # Update
        landmark_3D_new = P[:3]
        reference_2D_new = new_2D.copy()
        #################
        ###Triangulate###
        #################

        #Project the points from camera to world coordinates
        reference_2D = reference_2D_new.astype('float32')

        #landmark_3D = landmark_3D_new.T.copy()
        landmark_3D = inv_transform.dot(np.vstack((landmark_3D_new, np.ones((1, landmark_3D_new.shape[1])))))
        landmark_3D = landmark_3D.T

        ##### ####################### #######
        ##### Done, Next image please #######
        ##### ###############################
        reference_img = curImage

        ##### ################################## #######
        ##### START OF Print and visualize stuff #######
        ##### ##########################################
        # draw images
        draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100);
        true_x, true_y = int(truePose[i][3]) + 300, 600-(int(truePose[i][11]) + 100)

        print("Our estimate", (draw_x, draw_y), type(draw_x), type(draw_y))
        print("True", (true_x, true_y), type(true_x), type(true_y))

        curError = np.sqrt(
            (tvec[0] - truePose[i][3]) ** 2 +
            (tvec[1] - truePose[i][7]) ** 2 +
            (tvec[2] - truePose[i][11]) ** 2)
        
        if (curError > maxError):
            maxError = curError

        print(tvec[0],tvec[1],tvec[2], rvec[0], rvec[1], rvec[2])
        print([truePose[i][3], truePose[i][7], truePose[i][11]])
        
        text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
                                                                           float(tvec[2]));
        cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2);
        cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2);
        cv2.rectangle(traj, (10, 30), (550, 50), (0, 0, 0), cv2.FILLED);
        cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8);

        h1, w1 = traj.shape[:2]
        h2, w2 = curImage.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
        vis[:h1, :w1, :3] = traj
        vis[:h2, w1:w1 + w2, :3] = np.dstack((np.dstack((curImage,curImage)),curImage))

        cv2.imshow("Trajectory", vis);
        k = cv2.waitKey(1) & 0xFF
        if k == 27: break


    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Maximum Error: ', maxError)
    ##### ################################ #######
    ##### END OF Print and visualize stuff #######
    ##### ########################################

if __name__ == '__main__':
    left_img = getLeftImage(0)
    right_img = getRightImage(0)

    K = getK()

    playImageSequence(left_img, right_img, K)

image:  0
-------------------------------------------------
[[-12.75567397  -1.2172777   16.52341596]
 [-13.15374105  -1.33317458  17.12427972]
 [-12.77671008  -1.12081594  16.67096513]
 ...
 [  3.49392126   0.86441859   4.14729074]
 [  3.45830852  -0.88144306   4.07231213]
 [  3.50182787   0.1092062    4.103894  ]]
-------------------------------------------------
Our estimate (300, 500) <class 'int'> <class 'int'>
True (300, 500) <class 'int'> <class 'int'>
[-0.00076] [-0.00077691] [-0.00040673] [-8.34502269e-05] [-5.48880132e-05] [5.67321315e-05]
[5.551115e-17, 3.330669e-16, -4.440892e-16]
image:  1
-------------------------------------------------
[[-24.42872129  -2.59733287  29.07838205]
 [-23.14336407  -1.79728767  27.57618904]
 [-23.14336407  -1.79728767  27.57618904]
 ...
 [-10.41003553  -0.32540472 -12.1989066 ]
 [ -9.12525552   2.12508852 -10.48414673]
 [ -9.19187295   2.02441214 -10.52134473]]
-------------------------------------------------
Our estimate (286, 505) <class '

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 1 has a wrong type
>  - Can't parse 'center'. Sequence item with index 1 has a wrong type


In [ ]:
def triangulation(K, baseline, cur_landmark_3D, cur_reference_2D):
            # project the feature points to 3D with triangulation
            #projection matrix for Left and Right Image
            M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
            M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

            p1_flip = np.vstack((cur_reference_2D.T, np.ones((1, cur_reference_2D.shape[0]))))
            p2_flip = np.vstack((cur_landmark_3D.T, np.ones((1, cur_landmark_3D.shape[0]))))

            P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

            # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
            P = P / P[3]
            land_points = P[:3]

            return land_points.T, cur_reference_2D



M_new = K.dot(inv_transform) # Projection matrix for new image

# Transpose and homogenousify it
reference_flip = np.vstack((reference_2D.T, np.ones((1, reference_2D.shape[0]))))
tracked_flip = np.vstack((tracked_2Dpoints.T, np.ones((1, tracked_2Dpoints.shape[0]))))

P = cv2.triangulatePoints(M_old, M_new, reference_flip[:2], tracked_flip[:2])
M_old = M_new.copy()
# Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
P = P / P[3]

# Update
landmark_3D_new = P[:3]
reference_2D_new = tracked_2Dpoints.copy()

    

# Challenge 
The current implementation only uses features computed at the current timestep. However, as we process more images we potentially have a lot of features from previous timesteps that are still valid. The challenge is to expand the `extract_keypoints_surf(..., refPoints)` function by giving it old reference points. You should then combine your freshly computed features with the old features and remove all duplicates. This requires you to keep track of old features and 3D points.

Hint 1: look in `helpers.py` for removing duplicates.

Hint 2: you are not interested in points that are behind you, so remember to remove points that are negative in the direction you move.